# はじめに
* DeepNeuralNetを、TensorFlow等のライブラリを用いずにスクラッチで実装していく。
* ここでは最終的に5層の、画像の判別機をつくる。（CNNは用いない）
* ある程度の知識は前提として進める。
* 量が多くなるため、随時更新。

## 重みの初期化
* 重みを初期化するメソッドを作成する
* DeepLearningで学習を進める前に、W,bを初期化する必要がある。
* Wは0で初期化すると、学習が進まない。
* 最適な初期値については、Xavier,Heの初期値、といったものがあるが、ここでは単純に微小なランダムの値とする。

In [1]:
import numpy as np
np.random.seed(1)

In [2]:
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

* 上記は`layer_dims`に作成するDLの各層のユニット数を指定することで、Wとbを返してくれる。
* 例えば、inputは5、outputは1、隠れ層1層でユニット数3とすると下記。

In [3]:
initialize_parameters([5,3,1])

{'W1': array([[ 0.01624345, -0.00611756, -0.00528172, -0.01072969,  0.00865408],
        [-0.02301539,  0.01744812, -0.00761207,  0.00319039, -0.0024937 ],
        [ 0.01462108, -0.02060141, -0.00322417, -0.00384054,  0.01133769]]),
 'W2': array([[-0.01099891, -0.00172428, -0.00877858]]),
 'b1': array([[ 0.],
        [ 0.],
        [ 0.]]),
 'b2': array([[ 0.]])}

* W1,W2,b1,b2を作成してくれる。

* 続きます

## 参考
* [Coursera deeplearning.ai](https://www.coursera.org/learn/neural-networks-deep-learning/home/welcome)